# Preparation

In [ ]:
"""
# Download model
huggingface-cli download tiiuae/Falcon3-1B-Instruct-1.58bit --local-dir ~/models/tiiuae/Falcon3-1B-Instruct-1.58bit
# Compile
python setup_env.py -md ~/models/tiiuae/Falcon3-1B-Instruct-1.58bit -q i2_s

#.build/bin/llama-server -m /home/pathfinder/models/tiiuae/Falcon3-1B-Instruct-1.58bit/ggml-model-i2_s.gguf --host 127.0.0.1 --port 8080

# Download Falcon3-1B-Instruct
huggingface-cli download tiiuae/Falcon3-1B-Instruct --local-dir ~/models/tiiuae/Falcon3-1B-Instruct
"""

# Importing Libraries

In [ ]:
import os
from dataclasses import dataclass
from utils import set_seed, BitNet

# Configuration

In [ ]:
@dataclass
class CONFIG:
    # Debug
    debug: bool = False
    verbose: bool = True

    # Model
    model_id: str = "tiiuae/Falcon3-1B-Instruct"
    model_path: str = "/home/pathfinder/models/tiiuae/Falcon3-1B-Instruct-1.58bit/ggml-model-f32.gguf"
    quantized_path: str = "/home/pathfinder/models/tiiuae/Falcon3-1B-Instruct-1.58bit/ggml-model-i2_s.gguf"
    ctx_size: int = 1024

    # Generation
    max_new_tokens: int = 256
    ## Speculative Decoding
    num_assistant_tokens: int = 5
    assistant_confidence_threshold: float = 0.4

    # Device
    n_threads: int = 12

    # Seed
    seed = 42

config = CONFIG()

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
set_seed(config.seed)

In [ ]:
system_prompt = "You are an helpful assistant."
user_prompt = "Explain quantum mechanics in detail please."
assistant_response = "Quantum mechanics is a fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles, such as electrons, protons, and photons."

# Model

In [ ]:
bitnet = BitNet(
    model_id=config.model_id,
    quantized_path=config.quantized_path,
    host="127.0.0.1",
    port=8080,
    ctx_size=config.ctx_size,
    n_threads=config.n_threads,
    n_gpu_layers=0,
    batch_size=1,
    slot_id=1
)
bitnet.start_server(verbose=False)
bitnet.init_tokenizer(verbose=False)
bitnet.init_model(verbose=True)

In [ ]:
bitnet.speculative_decoding(
    text=bitnet.format_falcon_prompt(
        system_prompt=system_prompt,
        user_prompt=user_prompt
    ),
    max_new_tokens=100,
    num_assistant_tokens=5,
    confidence_threshold=0.22,
    verbose=True
)

# Generation

In [ ]:
bitnet.generate_hf(
    text=bitnet.format_falcon_prompt(
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        assistant_response=assistant_response
    ),
    max_new_tokens=100,
    verbose=True
)

In [ ]:
bitnet.generate_gguf(
    text=bitnet.format_falcon_prompt(
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        assistant_response=assistant_response
    ),
    max_new_tokens=100,
    verbose=True
)

## Speculative Decoding